This notebook converts single channel grayscale images to 5 channel images by adding blank channels. 
This is done to make the images compatible with the pre-trained models that expect 5 channel images. 
The code in this notebook will need to change to match a unique dataset, regretfully.    

Note that the data used here has four channels, but the model needs 5 channels input.

In [1]:
import multiprocessing as mp
import os
import pathlib
import shutil
import sys
from typing import List, Optional, Tuple

import cv2

# show the image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tifffile as tiff
import tqdm

try:
    cfg = get_ipython().config
    in_notebook = True
except NameError:
    in_notebook = False

In [2]:
def get_crop_counts(list_of_counts: List[Tuple[int, int, int]]) -> Tuple[int, int, int]:
    """
    Get the total counts of successful crops and ommited crops

    Parameters
    ----------
    list_of_counts : List[Tuple[int, int, int]]
        A list of tuples containing the counts of successful crops and ommited crops

    Returns
    -------
    Tuple[int, int, int]
        A tuple containing the total counts of successful crops and ommited crops
    """
    total_ommited = 0
    total_sucessful = 0
    total_total = 0
    for ommited, sucessful, total in list_of_counts:
        total_ommited += ommited
        total_sucessful += sucessful
        total_total += total
    assert total_total == total_ommited + total_sucessful
    return (total_ommited, total_sucessful, total_total)


def crop_image(
    metadata_df: pd.DataFrame,
    i: int,
    image_path: str,
    radius: int = 50,
    add_channels: Optional[bool] = False,
    total_channels: int = 5,
) -> Tuple[int, int, int]:
    """

    Crop the image based on the metadata and save the cropped image to disk
    Also output extracted metadata for the cropped image

    Parameters
    ----------
    i : int
        This is the iterator index for the metadata_df
    image_path : str
        Path to the image directory
    radius : int, optional
        The radius to crop the image by, by default 50
    add_channels : Optional[bool], optional
        This is a bool argument if set True will add extra channels to add up to 5 total , by default False

    Returns
    -------
    Tuple[int, int, int]
        A tuple containing the counts of omitted crops, successful crops and total crops
    """
    successful_count = 0
    omitted_count = 0
    total_count = 1
    image_information_df = metadata_df.copy().iloc[i]

    center_y = image_information_df["Metadata_Nuclei_Location_Center_Y"].astype(int)
    center_x = image_information_df["Metadata_Nuclei_Location_Center_X"].astype(int)
    well_fov = image_information_df["Metadata_Well_FOV"]
    image_path = pathlib.Path(f"{str(image_path)}{well_fov}").resolve(strict=True)
    # DNA
    image_name_DNA = pathlib.Path(image_information_df["Metadata_Image_FileName_DNA"])
    image_path_DNA = pathlib.Path(image_path / image_name_DNA).resolve(strict=True)
    # 488_1
    image_name_488_1 = pathlib.Path(
        image_information_df["Metadata_Image_FileName_CL_488_1"]
    )
    image_path_488_1 = pathlib.Path(image_path / image_name_488_1).resolve(strict=True)
    # 488_2
    image_name_488_2 = pathlib.Path(
        image_information_df["Metadata_Image_FileName_CL_488_2"]
    )
    image_path_488_2 = pathlib.Path(image_path / image_name_488_2).resolve(strict=True)
    # 561
    image_name_561 = pathlib.Path(
        image_information_df["Metadata_Image_FileName_CL_561"]
    )
    image_path_561 = pathlib.Path(image_path / image_name_561).resolve(strict=True)
    image_DNA = tiff.imread(image_path_DNA)
    image_488_1 = tiff.imread(image_path_488_1)
    image_488_2 = tiff.imread(image_path_488_2)
    image_561 = tiff.imread(image_path_561)

    image_DNA_crop = image_DNA[
        center_y - radius : center_y + radius, center_x - radius : center_x + radius
    ]
    image_488_1_crop = image_488_1[
        center_y - radius : center_y + radius, center_x - radius : center_x + radius
    ]
    image_488_2_crop = image_488_2[
        center_y - radius : center_y + radius, center_x - radius : center_x + radius
    ]
    image_561_crop = image_561[
        center_y - radius : center_y + radius, center_x - radius : center_x + radius
    ]

    # check if crop is an edge case
    # Where edge case is cells that are too close to the edge of the image to crop
    # This ensures that all crops are the same dimensions and can be used in the model
    if image_DNA_crop.shape[0] < radius * 2 or image_DNA_crop.shape[1] < radius * 2:
        omitted_count = 1
        return (omitted_count, successful_count, total_count)
    # merge the channels to a single image
    image_merge = np.stack(
        [image_DNA_crop, image_488_1_crop, image_488_2_crop, image_561_crop], axis=-1
    )
    if add_channels:
        if image_merge.shape[-1] < total_channels:
            channels_to_add = total_channels - image_merge.shape[-1]
            for _ in range(channels_to_add):
                # add a new channel of all zeros
                new_channels = np.zeros((image_merge.shape[0], image_merge.shape[1], 1))
                image_merge = np.concatenate((image_merge, new_channels), axis=-1)
    # save images to disk
    image_save_path = pathlib.Path(
        # f"../../../data/processed_images/crops/{image_information_df['Metadata_Well_FOV']}/time_{image_information_df['Metadata_Time']}_image_number_{image_information_df['Metadata_ImageNumber']}_cell_number_{image_information_df['Metadata_Nuclei_Number_Object_Number']}/"
        f"../data/processed_images/sc_crops/{image_information_df['Metadata_Well_FOV']}_time_{image_information_df['Metadata_Time']}_image_number_{image_information_df['Metadata_ImageNumber']}_cell_number_{image_information_df['Metadata_Nuclei_Number_Object_Number']}/"
    )
    image_save_path.mkdir(parents=True, exist_ok=True)
    file_name = image_information_df["Metadata_Image_FileName_DNA"].replace(
        ".tiff",
        f'image_number_{image_information_df["Metadata_ImageNumber"]}_cell_number_{image_information_df["Metadata_Nuclei_Number_Object_Number"]}_index_{i}_crop.tiff',
    )
    image_save_path = pathlib.Path(image_save_path / file_name)
    if os.path.exists(image_save_path):
        successful_count = 1
        return (omitted_count, successful_count, total_count)
    tiff.imwrite(image_save_path, image_merge)
    successful_count = 1
    return (omitted_count, successful_count, total_count)

## Import paths

In [ ]:
# set the path to the data directory
data_file_dir = pathlib.Path(
    "../../../6.process_CP_features/data/5.feature_select/profiles/features_selected_profile.parquet"
).resolve(strict=True)

# read in the data
cp_feature_data = pd.read_parquet(data_file_dir)
# print the data
print(cp_feature_data.shape)
pd.set_option("display.max_columns", None)
cp_feature_data.head()

(158431, 901)


,Metadata_plate,Metadata_Well,Metadata_number_of_singlecells,Metadata_compound,Metadata_dose,Metadata_control,Metadata_ImageNumber,Metadata_FOV,Metadata_Time,Metadata_Cells_Number_Object_Number,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,Metadata_ImageNumber_1,Metadata_ImageNumber_2,Metadata_ImageNumber_3,Metadata_Nuclei_Number_Object_Number,Metadata_Image_FileName_CL_488_1,Metadata_Image_FileName_CL_488_2,Metadata_Image_FileName_CL_561,Metadata_Image_FileName_DNA,Metadata_Image_PathName_CL_488_2,Metadata_Image_PathName_CL_561,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_coordinates_x,Metadata_track_id,Metadata_t,Metadata_y,Metadata_x,Metadata_id,Metadata_parent_track_id,Metadata_parent_id,Metadata_coordinates_y,Metadata_distance,Cytoplasm_AreaShape_Area,Cytoplasm_AreaShape_BoundingBoxArea,Cytoplasm_AreaShape_Center_X,Cytoplasm_AreaShape_Compactness,Cytoplasm_AreaShape_Eccentricity,Cytoplasm_AreaShape_Extent,Cytoplasm_AreaShape_FormFactor,Cytoplasm_AreaShape_MajorAxisLength,Cytoplasm_AreaShape_MeanRadius,Cytoplasm_AreaShape_Orientation,Cytoplasm_AreaShape_Solidity,Cytoplasm_AreaShape_Zernike_0_0,Cytoplasm_AreaShape_Zernike_1_1,Cytoplasm_AreaShape_Zernike_2_0,Cytoplasm_AreaShape_Zernike_2_2,Cytoplasm_AreaShape_Zernike_4_0,Cytoplasm_AreaShape_Zernike_4_2,Cytoplasm_AreaShape_Zernike_5_1,Cytoplasm_AreaShape_Zernike_6_0,Cytoplasm_AreaShape_Zernike_7_7,Cytoplasm_AreaShape_Zernike_8_4,Cytoplasm_Correlation_Correlation_CL_488_1_CL_488_2,Cytoplasm_Correlation_Correlation_CL_488_2_CL_561,Cytoplasm_Correlation_Correlation_CL_488_2_DNA,Cytoplasm_Correlation_Correlation_CL_561_DNA,Cytoplasm_Correlation_Costes_CL_488_2_CL_561,Cytoplasm_Correlation_Costes_DNA_CL_488_2,Cytoplasm_Correlation_Costes_DNA_CL_561,Cytoplasm_Correlation_K_CL_488_1_CL_488_2,Cytoplasm_Correlation_K_CL_488_1_CL_561,Cytoplasm_Correlation_K_CL_488_2_CL_488_1,Cytoplasm_Correlation_K_CL_488_2_DNA,Cytoplasm_Correlation_K_CL_561_CL_488_1,Cytoplasm_Correlation_K_CL_561_DNA,Cytoplasm_Correlation_K_DNA_CL_488_2,Cytoplasm_Correlation_K_DNA_CL_561,Cytoplasm_Correlation_Manders_CL_561_DNA,Cytoplasm_Correlation_Manders_DNA_CL_488_2,Cytoplasm_Correlation_Manders_DNA_CL_561,Cytoplasm_Correlation_Overlap_CL_488_1_CL_488_2,Cytoplasm_Correlation_Overlap_CL_488_1_DNA,Cytoplasm_Correlation_Overlap_CL_488_2_CL_561,Cytoplasm_Correlation_Overlap_CL_488_2_DNA,Cytoplasm_Correlation_Overlap_CL_561_DNA,Cytoplasm_Correlation_RWC_CL_488_1_CL_488_2,Cytoplasm_Correlation_RWC_CL_488_2_CL_488_1,Cytoplasm_Correlation_RWC_CL_488_2_CL_561,Cytoplasm_Correlation_RWC_CL_561_CL_488_2,Cytoplasm_Granularity_1_CL_488_1,Cytoplasm_Granularity_1_CL_488_2,Cytoplasm_Granularity_1_CL_561,Cytoplasm_Granularity_1_DNA,Cytoplasm_Intensity_IntegratedIntensityEdge_CL_488_2,Cytoplasm_Intensity_IntegratedIntensityEdge_CL_561,Cytoplasm_Intensity_IntegratedIntensityEdge_DNA,Cytoplasm_Intensity_IntegratedIntensity_CL_488_2,Cytoplasm_Intensity_IntegratedIntensity_CL_561,Cytoplasm_Intensity_IntegratedIntensity_DNA,Cytoplasm_Intensity_LowerQuartileIntensity_CL_488_2,Cytoplasm_Intensity_LowerQuartileIntensity_CL_561,Cytoplasm_Intensity_MADIntensity_CL_488_2,Cytoplasm_Intensity_MADIntensity_CL_561,Cytoplasm_Intensity_MaxIntensityEdge_CL_488_2,Cytoplasm_Intensity_MaxIntensityEdge_CL_561,Cytoplasm_Intensity_MaxIntensity_CL_488_2,Cytoplasm_Intensity_MeanIntensityEdge_CL_561,Cytoplasm_Intensity_MeanIntensityEdge_DNA,Cytoplasm_RadialDistribution_FracAtD_CL_561_1of4,Cytoplasm_RadialDistribution_FracAtD_DNA_4of4,Cytoplasm_RadialDistribution_MeanFrac_CL_561_1of4,Cytoplasm_RadialDistribution_RadialCV_DNA_1of4,Cytoplasm_RadialDistribution_RadialCV_DNA_2of4,Cytoplasm_RadialDistribution_RadialCV_DNA_3of4,Cytoplasm_RadialDistribution_ZernikeMagnitude_CL_488_2_1_1,Cytoplasm_RadialDistribution_ZernikeMagnitude_CL_488_2_2_0,Cytoplasm_RadialDistribution_ZernikeMagnitude_CL_488_2_2_2,Cytoplasm_RadialDistribution_ZernikeMagnitude_CL_488_2_3_1,Cytoplasm_RadialDistribution_ZernikeMagnitude_CL_488_2_3_

In [4]:
well_fov = cp_feature_data["Metadata_Well"] + "_F" + cp_feature_data["Metadata_FOV"]
cp_feature_data.insert(0, "Metadata_Well_FOV", well_fov)
# get columns that contain Metadata
metadata_columns = [col for col in cp_feature_data.columns if "Metadata" in col]
metadata_df = cp_feature_data[metadata_columns]
# get columns that contain Features
feature_df = cp_feature_data.drop(columns=metadata_columns)
# show all columns
metadata_df.head()

,Metadata_Well_FOV,Metadata_plate,Metadata_Well,Metadata_number_of_singlecells,Metadata_compound,Metadata_dose,Metadata_control,Metadata_ImageNumber,Metadata_FOV,Metadata_Time,Metadata_Cells_Number_Object_Number,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,Metadata_ImageNumber_1,Metadata_ImageNumber_2,Metadata_ImageNumber_3,Metadata_Nuclei_Number_Object_Number,Metadata_Image_FileName_CL_488_1,Metadata_Image_FileName_CL_488_2,Metadata_Image_FileName_CL_561,Metadata_Image_FileName_DNA,Metadata_Image_PathName_CL_488_2,Metadata_Image_PathName_CL_561,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_coordinates_x,Metadata_track_id,Metadata_t,Metadata_y,Metadata_x,Metadata_id,Metadata_parent_track_id,Metadata_parent_id,Metadata_coordinates_y,Metadata_distance,Metadata_Nuclei_AreaShape_BoundingBoxArea
0,C-10_F0003,1,C-10,150,Staurosporine,78.13,test,11,0003,10.0,1.0,1,5,11,11.0,11.0,5.0,C-10_F0003_T0011_Z0001_C02_illumcorrect.tiff,C-10_F0003_T0011_Z0001_C03_illumcorrect.tiff,C-10_F0003_T0011_Z0001_C04_illumcorrect.tiff,C-10_F0003_T0011_Z0001_C01_illumcorrect.tiff,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...,183.832252,54.943003,"[183.83225208526414, 54.94300278035218]",2,10.0,55.0,184.0,11000005.0,-1,10000004.0,"[184.0, 55.0]",0.177167,-0.276099
1,C-10_F0003,1,C-10,150,Staurosporine,78.13,test,11,0003,10.0,8.0,8,2,11,11.0,11.0,2.0,C-10_F0003_T0011_Z0001_C02_illumcorrect.tiff,C-10_F0003_T0011_Z0001_C03_illumcorrect.tiff,C-10_F0003_T0011_Z0001_C04_illumcorrect.tiff,C-10_F0003_T0011_Z0001_C01_illumcorrect.tiff,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...,1861.750943,11.265409,"[1861.7509433962264, 11.265408805031447]",153,10.0,11.0,1862.0,11000002.0,-1,10000001.0,"[1862.0, 11.0]",0.363966,-1.850673
2,C-10_F0003,1,C-10,150,Staurosporine,78.13,test,11,0003,10.0,15.0,15,13,11,11.0,11.0,13.0,C-10_F0003_T0011_Z0001_C02_illumcorrect.tiff,C-10_F0003_T0011_Z0001_C03_illumcorrect.tiff,C-10_F0003_T0011_Z0001_C04_illumcorrect.tiff,C-10_F0003_T0011_Z0001_C01_illumcorrect.tiff,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...,442.452953,163.574302,"[442.45295262816353, 163.5743024010383]",3,10.0,164.0,442.0,11000013.0,-1,10000013.0,"[442.0, 164.0]",0.621598,-0.701012
3,C-10_F0003,1,C-10,150,Staurosporine,78.13,test,11,0003,10.0,16.0,16,7,11,11.0,11.0,7.0,C-10_F0003_T0011_Z0001_C02_illumcorrect.tiff,C-10_F0003_T0011_Z0001_C03_illumcorrect.tiff,C-10_F0003_T0011_Z0001_C04_illumcorrect.tiff,C-10_F0003_T0011_Z0001_C01_illumcorrect.tiff,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...,1609.018100,91.010558,"[1609.0180995475114, 91.0105580693816]",144,10.0,91.0,1609.0,11000007.0,-1,10000005.0,"[1609.0, 91.0]",0.020954,-1.941156
4,C-10_F0003,1,C-10,150,Staurosporine,78.13,test,11,0003,10.0,23.0,23,15,11,11.0,11.0,15.0,C-10_F0003_T0011_Z0001_C02_illumcorrect.tiff,C-10_F0003_T0011_Z0001_C03_illumcorrect.tiff,C-10_F0003_T0011_Z0001_C04_illumcorrect.tiff,C-10_F0003_T0011_Z0001_C01_illumcorrect.tiff,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...,/gpfs/alpine1/scratch/mlippincott@xsede.org/li...,1875.239910,195.170404,"[1875.2399103139014, 195.17040358744396]",145,10.0,195.0,1875.0,11000015.0,-1,10000014.0,"[1875.0, 195.0]",0.294269,-1.683014


This cell is not run as it takes a long time to run...

In [5]:
image_path = pathlib.Path(
    "../../../2.cellprofiler_ic_processing/illum_directory/timelapse/20231017ChromaLive_6hr_4ch_MaxIP_"
)
radius = 50

In [6]:
# set the number of processes to use
if in_notebook:
    num_processes = mp.cpu_count() - 8
else:
    num_processes = mp.cpu_count()
print(f"Number of processes: {num_processes}")
# get the total number of rows in the metadata_df
total_crops = metadata_df.shape[0]
print(f"There are {total_crops:,} images to crop")

process_list = [
    mp.Process(target=crop_image, args=(i, image_path, radius, False))
    for i in range(total_crops)
]
print(f"There are {len(process_list):,} processes to run")
pool = mp.Pool(num_processes)
# run the processes in the pool with multiple args
results = pool.starmap_async(
    crop_image,
    [(metadata_df, i, image_path, radius, False) for i in range(total_crops)],
)
pool.close()
pool.join()
pool.terminate()
results = results.get()
final_counts = get_crop_counts(results)
# print the totals with commas for easy reading
print(
    f"Total crops: {final_counts[2]:,}, Sucessful crops: {final_counts[1]:,}, Ommited crops: {final_counts[0]:,}"
)

Number of processes: 16
There are 158,431 images to crop
There are 158,431 processes to run
Total crops: 158,431, Sucessful crops: 140,235, Ommited crops: 18,196
